## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [0]:
#from google.colab import drive

In [2]:
#drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [20]:
import pandas as pd
labels=pd.read_csv('labels.csv')

### Print the count of each category of Dogs given in the dataset



In [22]:
labels['breed'].value_counts()

scottish_deerhound                126
maltese_dog                       117
afghan_hound                      116
entlebucher                       115
bernese_mountain_dog              114
shih-tzu                          112
pomeranian                        111
great_pyrenees                    111
basenji                           110
samoyed                           109
airedale                          107
tibetan_terrier                   107
cairn                             106
leonberg                          106
beagle                            105
japanese_spaniel                  105
australian_terrier                102
miniature_pinscher                102
blenheim_spaniel                  102
irish_wolfhound                   101
lakeland_terrier                   99
saluki                             99
papillon                           96
whippet                            95
norwegian_elkhound                 95
siberian_husky                     95
pug         

In [4]:
labels['rank']=labels['breed'].rank(method='dense').astype(int) - 1


### Get one-hot encodings of labels

In [6]:
import keras 
#y_train = keras.utils.to_categorical(labels['rank'], 120)


C:\Users\Alekh\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
labels.shape

(10222, 3)

In [8]:
labels.head()

,id,breed,rank
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,19
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,37
2,001cdf01b096e06d78e9e5112d419397,pekinese,85
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,15
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,49


## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [9]:
img_rows=128
img_cols=128

from tqdm import tqdm
import cv2


In [9]:
x_feature = []
y_feature = []
i = 0 # initialisation
for f,img,rk in tqdm(labels.values): # f for format ,jpg
    img = cv2.imread('./train/{}.jpg'.format(f), 1)
    img_resize = cv2.resize(img, (img_rows, img_cols)) 
    x_feature.append(img_resize)
    y_feature.append(rk)

100%|████████████████████████████████████████████████████████████████████████████| 10222/10222 [02:18<00:00, 73.58it/s]


In [10]:
#labels.values

In [14]:
x_feature[1].shape

(128, 128, 3)

In [15]:
#x_feature[0:2]

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [16]:
import numpy as np
x_feature=np.asarray(x_feature)
y_feature=np.asarray(y_feature)


In [19]:
np.save('x_feature.out', x_feature)
np.save('y_feature.out', y_feature)

In [20]:
#y_feature.shape

In [21]:
#x_feature.shape

In [1]:
import numpy as np
x_feature=np.load('x_feature.out.npy')
y_feature=np.load('y_feature.out.npy')

In [2]:
x_feature=x_feature/255

In [3]:
import keras
y_feature = keras.utils.to_categorical(y_feature, 120)

C:\Users\Alekh\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [4]:
from sklearn.model_selection import train_test_split


x_train,x_val,y_train,y_val=train_test_split(x_feature,y_feature,test_size=0.1)

In [5]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(9199, 128, 128, 3)
(9199, 120)
(1023, 128, 128, 3)
(1023, 120)


### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [26]:
test_img=pd.read_csv('sample_submission.csv')
test_img=test_img['id']

Run the below code to load the test image files in x_test_feature

In [27]:
x_test_feature = []
i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f), 1)
    img_resize = cv2.resize(img, (img_rows, img_cols)) 
    x_test_feature.append(img_resize)

100%|████████████████████████████████████████████████████████████████████████████| 10357/10357 [04:07<00:00, 41.93it/s]


Normalize the test data and convert it into 4 dimensions

In [ ]:
x_test_feature=np.asarray(x_feature)
x_test_feature=x_test_feature/255
#x_test_feature = x_test_feature.astype('float32')
x_test_feature.shape

In [30]:
np.save('x_test_feature.out', x_test_feature)

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Conv2D

In [10]:
TRAIN = True
img_rows=128
img_cols=128

if TRAIN:
    input_shape = (img_rows,img_cols,3)
    print(input_shape)
    EPOCHS=10
    BATCH_SIZE = 128
    
    #Initialize the model
    model = Sequential()
    
    #Add a Convolutional Layer with 32 filters of size 3X3 and activation function as 'ReLU' 
    model.add(Conv2D(32, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=input_shape,name='conv_1'))
    
    #Add a Convolutional Layer with 32 filters of size 3X3 and activation function as 'ReLU' 
    model.add(Conv2D(32, (3, 3), activation='relu',name='conv_2'))
    
    #Flatten the layer
    model.add(Flatten())
    
    
    #Add Fully Connected Layer with 128 units and activation function as 'ReLU'
    model.add(Dense(256, activation='relu',name='dense_1',kernel_initializer='glorot_normal'))
    
    
    #Add Fully Connected Layer with 10 units and activation function as 'softmax'
    model.add(Dense(120, activation='softmax',name='dense_2',kernel_initializer='glorot_normal'))
    
    from keras.optimizers import Adam
    from keras.losses import categorical_crossentropy
    
    #To use adam optimizer for learning weights with learning rate = 0.001
    optimizer = Adam(lr=0.001)
    #Set the loss function and optimizer for the model training
    model.compile(loss=categorical_crossentropy,
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    # Store Training Results
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
    callback_list = [early_stopping]# [stats, early_stopping]
        
    # Train the model
    model.fit(x_train, y_train, epochs=EPOCHS,validation_data=(x_val, y_val)
              ,callbacks=callback_list 
              #,verbose=True
             )
    
    model.save('./CV_dog_breed.h5')
    
else:
    print('Loading pretrained model...')
    model1 = keras.models.load_model('./CV_dog_breed.h5')
    print('Model Loaded.')

(128, 128, 3)
Train on 9199 samples, validate on 1023 samples
Epoch 1/10
9199/9199 [==============================] - 2123s 231ms/step - loss: 4.9186 - acc: 0.0118 - val_loss: 4.7593 - val_acc: 0.0156
Epoch 2/10
9199/9199 [==============================] - 1706s 185ms/step - loss: 4.3712 - acc: 0.0998 - val_loss: 4.8344 - val_acc: 0.0342
Epoch 3/10
9199/9199 [==============================] - 1633s 178ms/step - loss: 1.3249 - acc: 0.7075 - val_loss: 8.6750 - val_acc: 0.0274
Epoch 4/10
9199/9199 [==============================] - 1633s 178ms/step - loss: 0.1410 - acc: 0.9788 - val_loss: 11.4059 - val_acc: 0.0293
Epoch 5/10
9199/9199 [==============================] - 1641s 178ms/step - loss: 0.0692 - acc: 0.9926 - val_loss: 10.8408 - val_acc: 0.0254
Epoch 6/10
9199/9199 [==============================] - 1638s 178ms/step - loss: 0.0449 - acc: 0.9948 - val_loss: 11.2080 - val_acc: 0.0196
Epoch 7/10
9199/9199 [==============================] - 1646s 179ms/step - loss: 0.0335 - acc: 0.9961

In [ ]:
#loss_and_metrics = model1.evaluate(x_test, y_test)
#print(loss_and_metrics)

### Use batch_size = 128 and epochs = 10 and execute the model

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    rotation_range=50,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1  # randomly shift images vertically (fraction of total height)
      # randomly flip images
    )  # randomly flip images

# Prepare the generator
datagen.fit(x_train)

# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=128),
                    steps_per_epoch=len(x_train) / 32, 
                    epochs=EPOCHS,callbacks=callback_list )

Epoch 1/10
  1/287 [..............................] - ETA: 112:34:06 - loss: 10.1421 - acc: 0.0156

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [4]:
from keras import applications
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [10]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, input_shape = (128, 128, 3))

Print the summary of the base_model

In [11]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [15]:
from keras import applications
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD, Adam

optimizer = Adam(lr=0.001)

for layer in base_model.layers:
    layer.trainable = False
    
    
    
    
    
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(120, activation="softmax")(x)

# creating the final model 
model_final = Model(input = base_model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics=["accuracy"])

    

C:\Users\Alekh\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [16]:
import numpy as np
x_feature=np.load('x_feature.out.npy')
y_feature=np.load('y_feature.out.npy')

x_feature=x_feature/255

import keras
y_feature = keras.utils.to_categorical(y_feature, 120)

from sklearn.model_selection import train_test_split


x_train,x_val,y_train,y_val=train_test_split(x_feature,y_feature,test_size=0.1)

In [17]:
# Save the model 
checkpoint = ModelCheckpoint("vgg16_best.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')

epochs=20
# Train the model 
model_final.fit(x_train, y_train, epochs = epochs, validation_data=(x_val, y_val), callbacks = [checkpoint, early])

Train on 9199 samples, validate on 1023 samples
Epoch 1/20
9199/9199 [==============================] - 2647s 288ms/step - loss: 4.7164 - acc: 0.0208 - val_loss: 4.4406 - val_acc: 0.0411

Epoch 00001: val_acc improved from -inf to 0.04106, saving model to vgg16_best.h5
Epoch 2/20
9199/9199 [==============================] - 2633s 286ms/step - loss: 4.0089 - acc: 0.0871 - val_loss: 3.8175 - val_acc: 0.0968

Epoch 00002: val_acc improved from 0.04106 to 0.09677, saving model to vgg16_best.h5
Epoch 3/20
9199/9199 [==============================] - 2614s 284ms/step - loss: 3.3448 - acc: 0.1805 - val_loss: 3.5637 - val_acc: 0.1447

Epoch 00003: val_acc improved from 0.09677 to 0.14467, saving model to vgg16_best.h5
Epoch 4/20
9199/9199 [==============================] - 2626s 285ms/step - loss: 2.8649 - acc: 0.2623 - val_loss: 3.3821 - val_acc: 0.1838

Epoch 00004: val_acc improved from 0.14467 to 0.18377, saving model to vgg16_best.h5
Epoch 5/20
9199/9199 [==============================] -

In [18]:
model_final.save('./model_final.h5')

In [ ]:
#Model accuracy increased to 96% but val accuracy is still 20% which is still overfit
# Best Val accuracy is 22% at 68% training accuracy again model is overfit